In [1]:
!pip install roboflow

# Get Dataset

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="cJUunyRGd3YuAM3cnXbU")
project = rf.workspace("volleyballcourt").project("volleyball-court-keypoints-k6y7r")
version = project.version(1)
dataset = version.download("coco")
                

loading Roboflow workspace...
loading Roboflow project...


In [3]:
import os
import torch
from torchvision.datasets import CocoDetection
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as patches


In [4]:
class CustomTransform:
    def __call__(self, image, target):
        # Ensure image is a tensor
        if not isinstance(image, torch.Tensor):
            image = F.to_tensor(image)  # Convert to tensor if it's not already
        # Return the image and target (target remains unchanged)
        return image, target


In [5]:
class CustomCocoDataset(CocoDetection):
    def __init__(self, root, annFile, transforms=None):
        super().__init__(root, annFile)
        self.transforms = transforms

    def __getitem__(self, idx):
        # Get image and target from the parent class
        img, target = super().__getitem__(idx)

        # Format the target into PyTorch's expected format
        formatted_target = {
            "boxes": [],
            "labels": []
        }
        for obj in target:
            # Append bounding box (convert to [x_min, y_min, x_max, y_max])
            x, y, w, h = obj["bbox"]
            formatted_target["boxes"].append([x, y, x + w, y + h])
            # Append label
            formatted_target["labels"].append(obj["category_id"])

        # Convert to tensors
        formatted_target["boxes"] = torch.tensor(formatted_target["boxes"], dtype=torch.float32)
        formatted_target["labels"] = torch.tensor(formatted_target["labels"], dtype=torch.int64)

        # Apply transforms to both image and target
        if self.transforms:
            img, formatted_target = self.transforms(img, formatted_target)

        return img, formatted_target


In [6]:
# Dataset paths
train_dir = "./volleyball-court-keypoints-1/train"
train_annotations = "./volleyball-court-keypoints-1/train/_annotations.coco.json"

valid_dir = "./volleyball-court-keypoints-1/valid"
valid_annotations = "./volleyball-court-keypoints-1/valid/_annotations.coco.json"

# Define train and validation datasets
train_dataset = CustomCocoDataset(
    root=train_dir,
    annFile=train_annotations,
    transforms=CustomTransform()  # Use the custom transform
)

valid_dataset = CustomCocoDataset(
    root=valid_dir,
    annFile=valid_annotations,
    transforms=CustomTransform()  # Use the custom transform
)

# Define DataLoaders
train_loader = DataLoader(
    train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x))
)

valid_loader = DataLoader(
    valid_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x))
)


loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


# Define the model

In [7]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load the pretrained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Modify the classifier to match the number of classes (1 class + background)
num_classes = 2  # 1 for volleyball court + 1 background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

# Optimizer

In [8]:
import torch.optim as optim

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler (optional)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)




# Train

In [9]:
num_epochs = 20  # Number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0

    for i, (imgs, targets) in enumerate(train_loader):
        # Move images and targets to the device
        imgs = [img.to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(imgs, targets)  # Get the loss dictionary
        losses = sum(loss for loss in loss_dict.values())  # Sum all loss components

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # Track epoch loss
        epoch_loss += losses.item()

        # Print batch loss every 10 steps (optional)
        if (i + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Batch Loss: {losses.item():.4f}")

    # Step the learning rate scheduler after the epoch
    lr_scheduler.step()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Total Loss: {epoch_loss:.4f}")


Epoch [1/20], Step [10/87], Batch Loss: 0.2875
Epoch [1/20], Step [20/87], Batch Loss: 0.1469
Epoch [1/20], Step [30/87], Batch Loss: 0.1011
Epoch [1/20], Step [40/87], Batch Loss: 0.1446
Epoch [1/20], Step [50/87], Batch Loss: 0.1210
Epoch [1/20], Step [60/87], Batch Loss: 0.1177
Epoch [1/20], Step [70/87], Batch Loss: 0.0852
Epoch [1/20], Step [80/87], Batch Loss: 0.1038
Epoch [1/20], Total Loss: 14.8641
Epoch [2/20], Step [10/87], Batch Loss: 0.1529
Epoch [2/20], Step [20/87], Batch Loss: 0.1146
Epoch [2/20], Step [30/87], Batch Loss: 0.1271
Epoch [2/20], Step [40/87], Batch Loss: 0.1285
Epoch [2/20], Step [50/87], Batch Loss: 0.1217
Epoch [2/20], Step [60/87], Batch Loss: 0.1633
Epoch [2/20], Step [70/87], Batch Loss: 0.1666
Epoch [2/20], Step [80/87], Batch Loss: 0.2037
Epoch [2/20], Total Loss: 13.3819
Epoch [3/20], Step [10/87], Batch Loss: 0.1682
Epoch [3/20], Step [20/87], Batch Loss: 0.1646
Epoch [3/20], Step [30/87], Batch Loss: 0.1374
Epoch [3/20], Step [40/87], Batch Loss:

# Validation and Evaluation

## Validation Loss
